In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.metrics.summary_evaluation import SummaryEvaluation
from scripts.models.ltr_nn_tf.train import LTRNNTFTrain
from scripts.models.rank_with_model.rank_with_model import RankModel
%reload_ext autoreload

In [4]:
import pandas as pd

# NN

## Carga de datos

In [8]:
ltr_params = {
    'mode': 'tfidf',
    'count_vec_kwargs': {'strip_accents': 'unicode'},
    'target_metric': 'rouge',
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'drop_teams': True,
    'lemma': True,
    'train_perc': 0.7,
    'val_perc': 0.2
}
DENSE_LAYERS = [500, 250, 125, 60, 20]
N_DROPOUT = 4

model_params = {
    'dense_layers': DENSE_LAYERS,
    'dense_activation': ['relu'] * len(DENSE_LAYERS),
    'dropout': [0.3] * N_DROPOUT,
    'optimizer': 'adam'
}
opt_metric = 'mean_squared_error'
epochs = 25
batch_size = 128
# Not in config
shuffle = True
max_queue_size = 50

In [9]:
ltr_train = LTRNNTFTrain(ltr_params, epochs=epochs, batch_size=batch_size,
                         model_params=model_params,
                         opt_metric=opt_metric)

Adding 1 zeros so that params have the same length
Setting target metric to rouge
Setting mode to tfidf
Optimizing mean_squared_error with adam
Epochs: 25
Batch size: 128


In [10]:
ltr_train.experiment_id()

'4083eddade'

In [11]:
rank_model = RankModel(ltr_train, n=10)

Model already trained


In [8]:
rank_model.path

'/home/carlos/MasterDS/tfm/models/rank_with_model/57e17a45b6'

## Evaluación

In [12]:
sms_dict = {
    'sent_emb': 'distilroberta-base-paraphrase-v1'
}

In [13]:
evaluation = SummaryEvaluation(metric='sms', sms_dict=sms_dict)

Setting target metric to sms


### Sin procesar

In [14]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=False)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/254dd4d848/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade_avg.pickle


In [15]:
avg_scores_dict

0.0003127267260807779

### Con procesado

In [16]:
scores_dict, avg_scores_dict = evaluation.evaluate_rank(rank_model,
                                                   preprocess_text=True)

Ranked summaries already exist
Reading ranked summaries from /home/carlos/MasterDS/tfm/models/rank_with_model/254dd4d848/summaries.csv
Saving to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade_processed
Performing evaluation for 4523 articles
Writing to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade_processed.pickle
Writing avg to /home/carlos/MasterDS/tfm/data/metrics/summaries/sms/rank_with_model_ltr_nn_rouge-1_r_4083eddade_processed_avg.pickle


In [17]:
avg_scores_dict

8.005127759695741e-05